In [1]:
import json

In [175]:
class Node(json.JSONEncoder):
    def __init__(self, data):
        self.isRoot = False
        self.parent = None
        self.children = []
        self.data = data
        self.layer = None
        self.index = None
    
    def addChild(self, child):
        if child.layer != None and self.layer > child.layer:
            return False
        
        child.parent = self
        self.children.append(child)
        child.layer = self.layer + 1
        return True
    
    def makeRoot(self):
        self.layer = 0
        self.isRoot = True
    
    def stringify(self):
        return json.dumps(self.__dict__)
    
    def __repr__(self):
        return str(self.index)
    
    def getJSON(self, keyname):
        if keyname == 'children':
            return [int(repr(x)) for x in self.children]
        elif keyname == 'parent':
            return None if self.parent is None else int(repr(self.parent))
        else:
            return self.__dict__[keyname]
    
    def default(self, obj):
        return {x:self.getJSON(x) for x in self.__dict__.keys()}
        
class NodeList(json.JSONEncoder):
    def __init__(self, array=[]):
        for i,x in enumerate(array):
            x.index = i
        self.nodes = array
        
    def add(self, node):
        node.index = len(self.nodes)
        self.nodes.append(node)
        
    def remove(self, index):
        del self.nodes[index]
        
    def default(self, obj):
        return [x.default(x) for x in self.nodes]

In [189]:
functions = []

n = Node("f(x)=2x")
n.makeRoot()
x = Node("x")
n.addChild(x)

arr = NodeList(array=[n,x])
functions.append(arr.default(arr))

m = Node("g(x,y)=x^2+y^2")
m.makeRoot()



In [190]:
functions

[[{'children': [1],
   'data': 'f(x)=2x',
   'index': 0,
   'isRoot': True,
   'layer': 0,
   'parent': None},
  {'children': [],
   'data': 'x',
   'index': 1,
   'isRoot': False,
   'layer': 1,
   'parent': 0}]]

In [185]:
json.dumps([x.default(x) for x in functions])

'[[{"isRoot": true, "layer": 0, "index": 0, "data": "f(x)=2x", "parent": null, "children": [1]}, {"isRoot": false, "layer": 1, "index": 1, "data": "x", "parent": 0, "children": []}]]'

In [187]:
with open('functions.json', 'w+') as f:
    f.write(json.dumps([x.default(x) for x in functions]))